In [1]:
#model isinga 2D na prostokątnej siatce
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani

In [4]:
#losujemy spin, liczymy energie i prawdopod przerzucenia go i zmiane akceptujemy albo nie

k = 1
J = 1 #calka wymiany
T = 5
beta = 1/k/T

N = 32 #16x16 spinow w siatce
NT = 100000 #liczba krokow czasowych
snap_every = 1000 #co tyle animacji pokazuje??
N_snapshots = NT // snap_every # // robi dzielenie calkowite bo nie mozemy miec 3.5 zdjec :D


spins = np.random.randint(0,2,size = (N,N))*2-1 #tu losujemy zeby bylo -1 albo 1
#magnetyzacja
magnetyzacja = spins.sum()

def plot(spins):
    fig = plt.figure()
    axis = fig.add_subplot(111)
    axis.imshow(spins, cmap = 'Greys', interpolation = 'none', origin = 'lower')
    axis.set_title(T)
    plt.show()

def flip_spin(spins):
    """
    1. wybieramy losowo spin
        * x, y = random int[0,16]
    2. liczymy energie
        * E = - J spin[x, y] * suma po sasiadach z uwzglednieniem okresowych warunkow brzegowych z modulo z ujemnymi
    3. liczymy prawdopodobienstwo przerzucenia = exp(-2*E*beta)
    4. losujemy uniform(0,1)
    5. jesli boltzmann > uniform(0,1): przerzucamy
    6. losowo wybrany spin jest przerzucony do gory nogami z prawdopodobienstwem boltzmannowskim
    """
    global magnetyzacja
    x, y = np.random.randint(0, N, size = 2)
    E = J* 2 * spins[x, y]* (spins[(x+1)%N, y] + spins[(x-1)%N, y]+ spins[x, (y+1)%N]+ spins[x, (y-1)%N])
    boltzmann = np.exp(-E*beta)
    uniform = np.random.random()
    
    if uniform < boltzmann:
        zmiana_magnetyzacji = -2*spins[x,y]
        spins[x,y] = spins[x,y] * -1
        magnetyzacja += zmiana_magnetyzacji

spins_history = np.empty((N_snapshots, N, N))
mag_history = np.empty(NT)
#plot(spins)
for i in range(NT):
    if (i%snap_every == 0):
        spins_history[i//snap_every] = spins
    mag_history[i] = magnetyzacja
    flip_spin(spins)

def plot_mag():
    plt.plot(mag_history/N**2)
    plt.show()
    
def animacja(): #nie musi brac arguemntu
    fig = plt.figure()
    axis = fig.add_subplot(111)
    #zmiana 1 ponizej: IM = spins_history[0] zamiast spins
    IM = axis.imshow(spins_history[0], cmap = 'Greys', interpolation = 'none', origin = 'lower')
    # zmiana 2
    title = axis.set_title("T = {}, i = {}".format(T,0))
    
    def animate(i_snapshot):
        IM.set_array(spins_history[i_snapshot])
        axis.set_title("T = {}, i = {}".format(T,i_snapshot*snap_every))
        return [IM, title]
    
    animacja_obiekt = ani.FuncAnimation(fig, animate, frames = N_snapshots)
    plt.show()

animacja()
plot_mag()

In [81]:
import scipy.fftpack as fft
#y-ki sa zle zwrocone wiec trzeba zmienic
f = fft.rfftfreq(NT)
t = np.arange(10000)
y = np.sin(2*np.pi*t/1000)+np.sin(2*np.pi*t/(1000/10))
y_transf = fft.rfftfreq(10000)
y_fft = fft.rfft(y)**2
y_fft[0] = 5e8

y_odwrotna = fft.irfft(y_fft)/10000
#plt.plot(y_transf, y_fft, 'bo-')
plt.plot(t, y, 'b-')
plt.plot(t, y_odwrotna, 'r-')
plt.show()


plt.plot(f, np.log10(fft.rfft(mag_history)**2))
plt.show()
fft.rfft(mag_history) #real fast fourier transform

array([ -3.48987680e+07,  -2.08047292e+06,  -8.85596736e+06, ...,
         4.45479934e+02,  -1.33630714e+02,   4.84000000e+02])

In [88]:
#poisson 1D przez fouriera
N = 1000
x = np.linspace(-10,10,1000)
dx = x[1]-x[0]
#gestosc ladunku z potencjalu = gestosc ladunku/epsilon 0 (1)
#d2phi/dx2 = rho
#(d/dx) -> -i*k
#(d/dx)^2 -> -k^2
# -k^2 phi = rho
# phi = -rho/k^2

epsilon_0 = 1
rho = np.sin(x**2)

k = fft.rfftfreq(N, dx)
phi_f = -fft.rfft(rho)/k**2
phi_f[0] = 0
phi = fft.irfft(phi_f)/N #*normalizacja
plt.plot(x, rho)
plt.plot(x, phi)
plt.show()

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in true_divide
